In [1]:
# add src to the notebook path
import sys
sys.path.append("../src/")

# import from src
from utils import *
from models.model import *
from data.dataset import *
from constants import *


# import python libraries
import os
import PIL
import glob
import wandb
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torchinfo import summary


os.environ['WANDB_API_KEY'] = wandb_key
os.environ['WANDB_NOTEBOOK_NAME'] = os.getcwd()
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# Training

In [2]:
config = load_yaml_as_dict("../config/config.yaml")
config

{'data_path': '../data/mp3dlayout/validation',
 'image_size': 256,
 'model': {'config': 'model_config.yaml'},
 'training': {'epochs': 10,
  'lr': 0.002,
  'batch_size': 32,
  'shuffle': True,
  'loss_fn': 'mse',
  'optim': 'sgd',
  'scheduler': 'None'},
 'wandb': {'mode': 'disabled'}}

In [3]:
testing_images = config["data_path"]

image_size = (config["image_size"], config["image_size"])
transform = T.Compose([
        #T.ToPILImage(),
        #T.CenterCrop(0.75 * 64),
        T.Resize(image_size),
        #T.RandomResizedCrop(image_size),
        #T.RandomHorizontalFlip(),
        T.ToTensor()
        ])

train_set = MP3DL_Dataset(testing_images, transform, load_all=True)
train_loader = DataLoader(train_set, batch_size=config["training"]["batch_size"], shuffle=config["training"]["shuffle"])

  0%|          | 0/456 [00:00<?, ?it/s]

  0%|          | 0/456 [00:00<?, ?it/s]

In [4]:
model = MyModel()

summary(model)

Layer (type:depth-idx)                   Param #
MyModel                                  --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       12
│    └─SiLU: 2-2                         --
│    └─Conv2d: 2-3                       12
│    └─SiLU: 2-4                         --
│    └─Conv2d: 2-5                       4
│    └─SiLU: 2-6                         --
Total params: 28
Trainable params: 28
Non-trainable params: 0

In [5]:
loss_fn = select_loss_function(config["training"]["loss_fn"])
# Optimizers specified in the torch.optim package
optimizer = select_optimizer(model, config["training"]["optim"], learning_rate=config["training"]["lr"], momentum=0.9)
scheduler = None #select_scheduler(config["training"]["scheduler"], optimizer, last_epoch=-1, **kwargs)

In [6]:
# start a new wandb run to track this script
wandb.init(project="my-awesome-project", config=config, mode="online", settings=wandb.Settings(disable_job_creation=True)) # mode: [online, disabled]
    
# simulate training
epochs = config["training"]["epochs"]

model = model.to(device)
for epoch in tqdm(range(epochs)):
    loss = train_one_epoch(model, train_loader, loss_fn, optimizer, scheduler)
    #acc  = evaluate(model, validation_loader)
    # log metrics to wandb
    wandb.log({"loss": loss})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: Currently logged in as: mattiadurso98 (mattiaduso). Use `wandb login --relogin` to force relogin


  0%|          | 0/10 [00:00<?, ?it/s]

/home/mattia/miniconda3/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


loss,█▅▂▁▁▁▁▁▁▁
loss,0.03763
